In [1]:
import pandas as pd
import tensorflow
import numpy as np
import seaborn as sns

In [2]:
df2 = pd.read_csv('CSV/concrete.csv', header=None, skiprows=1)

In [3]:
df2.shape

(1030, 9)

In [4]:
df1 = df2.sample(frac=1)

In [6]:
train = int(df1.shape[0] / 100 * 70)
valdte = int(df1.shape[0] /100 * 20) + train
test = int(df1.shape[0] - valdte)

valdte + test

1030

In [7]:
df = df1.copy(deep=True)

Train_Data = df.iloc[:valdte, :8].to_numpy()
Train_Label = df.iloc[:valdte, -1].to_numpy()

Test_Data = df.iloc[valdte:, :8].to_numpy()
Test_Label = df.iloc[valdte:, -1].to_numpy()

print(Train_Data.shape)
print(Train_Label.shape)
print(Test_Data.shape)
print(Test_Label.shape)

mean = Train_Data.mean(axis=0)
Train_Data -= mean
std = Train_Data.std(axis=0)
Train_Data /= std
Test_Data -= mean
Test_Data /= std

Validte_Data = Train_Data[train:]
Validte_Label = Train_Label[train:]
Train_Data = Train_Data[:train]
Train_Label = Train_Label[:train]

print(f'\n{Train_Data.shape}')
print(Train_Label.shape)
print(Validte_Data.shape)
print(Validte_Label.shape)
print(Test_Data.shape)
print(Test_Label.shape)

(927, 8)
(927,)
(103, 8)
(103,)

(721, 8)
(721,)
(206, 8)
(206,)
(103, 8)
(103,)


In [8]:
from tensorflow.keras import models
from tensorflow.keras import layers

def build_model():
    model = models.Sequential()
    model.add(layers.Dense(12, activation='relu',
        input_shape=(Train_Data.shape[1],)))
    model.add(layers.Dense(12, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

In [9]:
model = build_model()
model.fit(Train_Data, Train_Label, epochs=13, validation_data=(Validte_Data, Validte_Label))
model.evaluate(Test_Data, Test_Label)

Train on 721 samples, validate on 206 samples
Epoch 1/13
721/721 [==============================] - 1s 2ms/sample - loss: 1501.0178 - mae: 34.9584 - val_loss: 1477.2298 - val_mae: 34.5810
Epoch 2/13
721/721 [==============================] - 0s 118us/sample - loss: 1464.2939 - mae: 34.4454 - val_loss: 1440.7906 - val_mae: 34.0659
Epoch 3/13
721/721 [==============================] - 0s 116us/sample - loss: 1424.2340 - mae: 33.8870 - val_loss: 1398.5770 - val_mae: 33.4631
Epoch 4/13
721/721 [==============================] - 0s 136us/sample - loss: 1378.9450 - mae: 33.2430 - val_loss: 1351.0364 - val_mae: 32.7744
Epoch 5/13
721/721 [==============================] - 0s 114us/sample - loss: 1325.7151 - mae: 32.4819 - val_loss: 1295.3239 - val_mae: 31.9491
Epoch 6/13
721/721 [==============================] - 0s 116us/sample - loss: 1265.0143 - mae: 31.5973 - val_loss: 1231.5561 - val_mae: 30.9961
Epoch 7/13
721/721 [==============================] - 0s 94us/sample - loss: 1197.3317 - mae

[633.2443071383874, 20.9841]

In [10]:
df = df1.copy(deep=True)

Train_Data = df.iloc[:valdte, :8].to_numpy()
Train_Label = df.iloc[:valdte, -1].to_numpy()

Test_Data = df.iloc[valdte:, :8].to_numpy()
Test_Label = df.iloc[valdte:, -1].to_numpy()

print(Train_Data.shape)
print(Train_Label.shape)
print(Test_Data.shape)
print(Test_Label.shape)

mean = Train_Data.mean(axis=0)
Train_Data -= mean
std = Train_Data.std(axis=0)
Train_Data /= std
Test_Data -= mean
Test_Data /= std

(927, 8)
(927,)
(103, 8)
(103,)


In [11]:
import numpy as np
k = 4
num_val_samples = len(Train_Data) // k
num_epochs = 100
all_scores = []

In [12]:
for i in range(k):
    print('processing fold #', i)
    val_data = Train_Data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = Train_Label[i * num_val_samples: (i + 1) * num_val_samples]
    partial_train_data = np.concatenate(
        [Train_Data[:i * num_val_samples],
        Train_Data[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [Train_Label[:i * num_val_samples],
        Train_Label[(i + 1) * num_val_samples:]],
        axis=0)
    model = build_model()
    model.fit(partial_train_data, partial_train_targets,
    epochs=num_epochs, batch_size=1, verbose=0)
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_mae)

processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3


In [13]:
np.mean(all_scores)

4.89936